In [1]:
import scipy
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

from sklearn import metrics
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('https://www4.stat.ncsu.edu/~boos/var.select/diabetes.tab.txt', sep='\t')

Y = data['Y']
X = data.drop(columns=['Y']) 
X = pd.get_dummies(X, columns=['SEX'])

idx = list(range(X.shape[0]))
train_idx, valid_idx = train_test_split(idx, test_size=0.3, random_state=2023)
print(">>>> # of Train data : {}".format(len(train_idx)))
print(">>>> # of valid data : {}".format(len(valid_idx)))

# Scaling
scaler = MinMaxScaler().fit(X.iloc[train_idx])
X_scaled = scaler.transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

>>>> # of Train data : 309
>>>> # of valid data : 133


In [3]:
def sse(clf, X, y):
    """Calculate the standard squared error of the model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    float
        The standard squared error of the model.
    """
    y_hat = clf.predict(X)
    sse = np.sum((y_hat - y) ** 2)
    return sse / X.shape[0]


def adj_r2_score(clf, X, y):
    """Calculate the adjusted :math:`R^2` of the model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    float
        The adjusted :math:`R^2` of the model.
    """
    n = X.shape[0]  # Number of observations
    p = X.shape[1]  # Number of features
    r_squared = metrics.r2_score(y, clf.predict(X))
    return 1 - (1 - r_squared) * ((n - 1) / (n - p - 1))


def coef_se(clf, X, y):
    """Calculate standard error for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of standard errors for the beta coefficients.
    """
    n = X.shape[0]
    X1 = np.hstack((np.ones((n, 1)), np.matrix(X)))
    se_matrix = scipy.linalg.sqrtm(
        metrics.mean_squared_error(y, clf.predict(X)) *
        np.linalg.inv(X1.T * X1)
    )
    return np.diagonal(se_matrix)


def coef_tval(clf, X, y):
    """Calculate t-statistic for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of t-statistic values.
    """
    a = np.array(clf.intercept_ / coef_se(clf, X, y)[0])
    b = np.array(clf.coef_ / coef_se(clf, X, y)[1:])
    return np.append(a, b)


def coef_pval(clf, X, y):
    """Calculate p-values for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of p-values.
    """
    n = X.shape[0]
    t = coef_tval(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def summary(clf, X, y, xlabels=None):
    """
    Output summary statistics for a fitted regression model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    xlabels : list, tuple
        The labels for the predictors.
    """
    # Check and/or make xlabels
    ncols = X.shape[1]
    if xlabels is None:
        xlabels = np.array(
            ['x{0}'.format(i) for i in range(1, ncols + 1)], dtype='str')
    elif isinstance(xlabels, (tuple, list)):
        xlabels = np.array(xlabels, dtype='str')
    # Make sure dims of xlabels matches dims of X
    if xlabels.shape[0] != ncols:
        raise AssertionError(
            "Dimension of xlabels {0} does not match "
            "X {1}.".format(xlabels.shape, X.shape))
    # Create data frame of coefficient estimates and associated stats
    coef_df = pd.DataFrame(
        index=['_intercept'] + list(xlabels),
        columns=['Estimate', 'Std. Error', 't value', 'p value']
    )
    try:
        coef_df['Estimate'] = np.concatenate(
            (np.round(np.array([clf.intercept_]), 6), np.round((clf.coef_), 6)))
    except Exception as e:
        coef_df['Estimate'] = np.concatenate(
            (
                np.round(np.array([clf.intercept_]), 6),
                np.round((clf.coef_), 6)
            ), axis = 1
    )[0,:]
    coef_df['Std. Error'] = np.round(coef_se(clf, X, y), 6)
    coef_df['t value'] = np.round(coef_tval(clf, X, y), 4)
    coef_df['p value'] = np.round(coef_pval(clf, X, y), 6)
    # Output results
    print('Coefficients:')
    print(coef_df.to_string(index=True))
    print('---')
    print('R-squared:  {0:.6f},    Adjusted R-squared:  {1:.6f},    MSE: {2:.1f}'.format(
        metrics.r2_score(y, clf.predict(X)), adj_r2_score(clf, X, y), sse(clf, X, y)))

In [4]:
penalties = [0.00001, 0.00005, 0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.6, 0.7, 0.9, 1, 10]
l1_ratio = [0.9, 0.7, 0.5, 0.3, 0.1]

for a in penalties:
    for b in l1_ratio:
        model = ElasticNet(alpha=a, l1_ratio=b).fit(X_scaled.iloc[train_idx], Y.iloc[train_idx]) 
        
        score = model.score(X_scaled.iloc[valid_idx], Y.iloc[valid_idx])
        pred_y = model.predict(X_scaled.iloc[valid_idx])
        mse = mean_squared_error(Y.iloc[valid_idx], pred_y)
        
        print(f"Alpha:{a:.7f}, l1_ratio: {b:.7f}, R2:{score:.7f}, MSE:{mse:.7f}, RMSE:{np.sqrt(mse):.7f}")

Alpha:0.0000100, l1_ratio: 0.9000000, R2:0.5301781, MSE:3084.5271311, RMSE:55.5385193
Alpha:0.0000100, l1_ratio: 0.7000000, R2:0.5302023, MSE:3084.3677950, RMSE:55.5370849
Alpha:0.0000100, l1_ratio: 0.5000000, R2:0.5302256, MSE:3084.2148395, RMSE:55.5357078
Alpha:0.0000100, l1_ratio: 0.3000000, R2:0.5302480, MSE:3084.0680163, RMSE:55.5343859
Alpha:0.0000100, l1_ratio: 0.1000000, R2:0.5302695, MSE:3083.9270883, RMSE:55.5331170
Alpha:0.0000500, l1_ratio: 0.9000000, R2:0.5302272, MSE:3084.2044920, RMSE:55.5356146
Alpha:0.0000500, l1_ratio: 0.7000000, R2:0.5303300, MSE:3083.5299323, RMSE:55.5295411
Alpha:0.0000500, l1_ratio: 0.5000000, R2:0.5304135, MSE:3082.9815630, RMSE:55.5246032
Alpha:0.0000500, l1_ratio: 0.3000000, R2:0.5304811, MSE:3082.5375315, RMSE:55.5206046
Alpha:0.0000500, l1_ratio: 0.1000000, R2:0.5305356, MSE:3082.1801728, RMSE:55.5173862
Alpha:0.0001000, l1_ratio: 0.9000000, R2:0.5302829, MSE:3083.8388547, RMSE:55.5323226
Alpha:0.0001000, l1_ratio: 0.7000000, R2:0.5304499, MS

In [5]:
grid = dict()
grid['alpha'] = penalties
grid['l1_ratio'] = l1_ratio

In [6]:
model = ElasticNet()
search = GridSearchCV(model, grid, scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1)

results = search.fit(X_scaled.iloc[valid_idx], Y.iloc[valid_idx])
print(f'RMSE: {-results.best_score_:.4f}')
print(f'Config: {results.best_params_}')

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.137e+05, tolerance: 6.790e+01
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.086e+05, tolerance: 7.254e+01
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.684e+04, toleranc

RMSE: 56.8906
Config: {'alpha': 0.01, 'l1_ratio': 0.3}


In [7]:
model_best = ElasticNet(alpha=results.best_params_['alpha'], l1_ratio=results.best_params_['l1_ratio']).fit(X_scaled.iloc[train_idx], Y.iloc[train_idx])

score = model_best.score(X_scaled.iloc[valid_idx], Y.iloc[valid_idx])
pred_y = model_best.predict(X_scaled.iloc[valid_idx])
mse = mean_squared_error(Y.iloc[valid_idx], pred_y)

print(f"Alpha:{results.best_params_['alpha']:.7f}, l1_ratio: {results.best_params_['l1_ratio']:.7f}, R2:{score:.7f}, MSE:{mse:.7f}, RMSE:{np.sqrt(mse):.7f}")
summary(model_best, X_scaled.iloc[valid_idx], Y.iloc[valid_idx], xlabels=X.columns)

Alpha:0.0100000, l1_ratio: 0.3000000, R2:0.5171797, MSE:3169.8655476, RMSE:56.3015590
Coefficients:
              Estimate                  Std. Error         t value   p value
_intercept   45.921027  3.856246e+08+1.830330e+00j  0.0000-0.0000j  1.000000
AGE          -7.326230  2.359368e+01+3.757100e-01j -0.3104+0.0049j  0.756688
BMI         112.053983  3.216612e+01+1.416070e-01j  3.4835-0.0153j  0.000672
BP           63.835355  2.827829e+01+3.470530e-01j  2.2571-0.0277j  0.025637
S1          -10.286770  1.657496e+02+2.931720e-01j -0.0621+0.0001j  0.950607
S2          -11.664902  1.168114e+02+2.728270e-01j -0.0999+0.0002j  0.920606
S3          -51.885195  7.360984e+01+6.006600e-02j -0.7049+0.0006j  0.482135
S4           26.375923  5.789022e+01+1.726370e-01j  0.4556-0.0014j  0.649414
S5          102.402575  5.069631e+01+2.943590e-01j  2.0199-0.0117j  0.045419
S6           25.791145  3.435827e+01+3.654500e-01j  0.7506-0.0080j  0.454223
SEX_1        10.231459  3.856246e+08-1.188440e-01j  0